<a href="https://colab.research.google.com/github/MathewzVk/llm-starter/blob/main/bygram_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
with open('/content/wizard_oz.txt', 'r', encoding='utf-8') as f:
  text=f.read()

print(len(text))

232308


In [127]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


# **Character level tokenizing**

In [128]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


In [129]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [130]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


# **Splitting Text**

In [131]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# **Input and target**

In [132]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y


x, y = get_batch('train')
print('input : ')
print(x)
print('targets : ')
print(y)

tensor([158296, 117231, 129930, 184751])
input : 
tensor([[67, 73,  1, 71, 68, 68, 59, 11],
        [73,  1, 67, 68, 73, 61, 62, 67],
        [69, 62, 67, 60,  1, 62, 73, 72],
        [76, 62, 67, 60,  1, 54,  1, 55]], device='cuda:0')
targets : 
tensor([[73,  1, 71, 68, 68, 59, 11],
        [ 1, 67, 68, 73, 61, 62, 67],
        [62, 67, 60,  1, 62, 73, 72],
        [62, 67, 60,  1, 54,  1, 55]], device='cuda:0')


**Batch size hyperparameter**

In [133]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [134]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            print("Logits size:", logits.size())
            print("Targets size:", targets.size())
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


lv4tW?:th!ywp
iC ]Nw[sEoUh_jzD5U_oa6DzR*zuSuUwPVhEwG''bJrv*)qICFr7lpaBLT)orlfIa]XYlfu)M!m]p![B9kdhLtPkDwa2Cp1
L*b8JOCS1e4E5U1X]5Pnt!(!,2_V*gon9!RH2nu6a[)6fTEj9)Vv*CKFCK!((M(8H3S]-hCe[Hct6z7G4_"mlrNO(r7;v*FhNw'xcr:d5DcW;4;v3Ry"sBNCmzfk3v09xRJirNg-hRpk;gla2PZX5U:VhjHRM;A;yia_j?doLdra)6uT
Y5bVy5b.gheOoNYtsKCLTp,E?XJWwva 
p6,hReklECtp5GcG.qucx&nRJlHs-1Hy8VvG9TA:[B*fCVBH_)L,fV6R;,3'ytQcbZGJRFwQsl!;a,IraTqS?p,KD_?X4f*E!Jm7b.BpKM7!D5U3j",E.D Tmz*-KY
_xbl.[spw'bhE& f0w;zPz6(Fu"y[[kI(,8p.(!dmSK0"s_G2j1DC


In [135]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f'step: {iter}, loss {losses}')

  xb, yb = get_batch('train')

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

tensor([46569, 67900,  5691, 12103])


RuntimeError: shape '[32]' is invalid for input of size 28

In [112]:
context = torch.zeros((1, 1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


]H:?S!kM*v,:MUXu))bV6)gsRrP?'gY'g2!Y aXW.9,xQwXsDwFRZq)O'h'x8*D
e
'iZHCYWIRmts"wX&F0o:&em]YAjq)E'JmJYy*8o.&GD*Ey)AcCkw''P H 26q!I82FG Ys!-vw8 SidI-4:?)pX:Nl?lp?VWz2II
0G"h.Fo,.jnmixOC4&GWNtMJb ;z
uKrbqF]-4q-vbLl,Yd0[;Lg)sCY]F9&oE02sdEEnJ7]_9uzT;Zod0Uc44ewX-dFg[6HdI8T;_bOF-r3[&V!,zr3)4v:0xQPP W7Mi3aZ9Idt4KZzzg3l?]SNx6jB23l3yOONTr2g.spFQ9rDAWXIBUgKU0v)7WOCB3AjCGAxx'Si*,t[M:'K9)x rT7lN 0D1U';XoO7t:LScmH,J78D&]Hlt7K;
kp7kMg2.)Y5Lm2w0x X&OGfYfV'?nm:azy9bJujeyAb]V(4XNo]y-MWNfcF94,Df"ScF*''lNmGzmQzQ.QK
